# An AS/AD-model for a small, open economy with fixed exchange rate

This project analyzes the effects of shocks to a simple AS/AD-model for a small, open economy in a fixed exchange rate regime. 

Imports and set magics:

In [1]:
import numpy as np 
from scipy import optimize
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display # For interactive figure
from ipywidgets import interact, IntSlider, Play, Dropdown

plt.rcParams.update({"axes.grid":True,"grid.color":"black","grid.alpha":"0.25","grid.linestyle":"--"})
plt.rcParams.update({'font.size': 14})

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# Plot settings:
plt.style.use('seaborn-whitegrid')
prop_cycle = plt.rcParams['axes.prop_cycle']

plt.rcParams["figure.autolayout"] = True
plt.rcParams["axes.edgecolor"] = "black"
plt.rcParams["axes.linewidth"] = 1.0
plt.rcParams["axes.xmargin"] = 0.01
plt.rcParams["axes.xmargin"] = 0.01
plt.rcParams["legend.edgecolor"] = "black"

# local modules
import modelproject
from modelproject import ASAD

OSError: 'seaborn-whitegrid' is not a valid package style, path of style file, URL of style file, or library style name (library styles are listed in `style.available`)

# Model description

**Presentation of model equations** 

Aggregated Demand and Aggregated Supply for a closed economy with stocastic and persistent shocks.

1. **x:** $$y_{t}-\overline{y}=\beta_1(e_{t-1}^r+\pi^f-\pi_t)-\beta_2(i^f-\pi_{t+1}^{e}-\overline{r}^f)+\tilde{z}_t$$
where
$$\tilde{z}_t=\beta_3(g_t-\overline{g})$$

2. **x:** $$g_t-\overline{g}=a(\overline{y}-y_t)$$

3. **x:** $$\pi_t=\pi^{e}_t+\gamma(y_t-\overline{y})+s_t$$

4. **x:** $$e^r_t=e^r_{t-1}+\pi^f-\pi_t$$

5. **x:** $$\pi_t^{e}=\pi^f_t$$

2. **Taylor-Rule:** $$i_{t}=\overline{r}+\pi_{t+1}^{e}+h\left(\pi_{t}-\pi^{*}\right)+b\left(y_{t}-\overline{y}\right)+\hat{\rho}_{t}$$
3. **AS Curve:** $$ \pi_{t}=\pi_{t}^{e}+\gamma(y_{t}-\overline{y})+s_{t} $$
4. **Static inflation expectations:** $$ \pi^e_t = \pi_{t-1}$$


Firstly we will define the AD curve as a combination of equation 1), 2) 3) & 5):
$$y_{t}-\overline{y}=-\alpha\left(\pi_{t}-\pi^{*}\right)+z_{t} $$
$$ \alpha=\frac{\alpha_{2}h}{1+\alpha_{2}b},z_{t}=\frac{\alpha_{1}(g_{t}-\overline{g})-\alpha_{2}\hat{\rho}-\alpha_{3}(\tau_{t}-\overline{\tau})+v_{t}}{1+\alpha_{2}b} $$

$$ \pi_{t}=\pi_{t-1}+\gamma(y_{t}-\overline{y})+s_{t} 

## Analytical solution

If your model allows for an analytical solution, you should provide here.

You may use Sympy for this. Then you can characterize the solution as a function of a parameter of the model.

To characterize the solution, first derive a steady state equation as a function of a parameter using Sympy.solve and then turn it into a python function by Sympy.lambdify. See the lecture notes for details. 

## Numerical solution

You can always solve a model numerically. 

Define first the set of parameters you need. 

Then choose one of the optimization algorithms that we have gone through in the lectures based on what you think is most fitting for your model.

Are there any problems with convergence? Does the model converge for all starting values? Make a lot of testing to figure these things out. 

# Further analysis

Make detailed vizualizations of how your model changes with parameter values. 

Try to make an extension of the model. 

# Conclusion

Add concise conclusion. 